In [1]:
import pandas as pd
import numpy as np
import fastf1
import fastf1.plotting
import matplotlib as mpl
from fastf1 import utils
import requests
from datetime import datetime, timedelta
import warnings
import math


In [2]:
#To remove warnings 
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
# Enable the cache
fastf1.Cache.enable_cache('cache') 
pd.options.mode.chained_assignment = None

## Variables

In [4]:
#TODO Check wich date to start 
year_start= 2018
year_finish= 2023

## Race seleccion 
- The first race of the 2023 season 

In [5]:
year = 2023
round = 1

In [6]:
session = fastf1.get_session(year, round, 'R')
session.load()

core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	No cached data found for car_data. Loading data...
api            INFO 	Fetching car data...
api            INFO 	Parsing car data...
api            INFO 	Data has been written to cache!
api            INFO 	No cached data found for position_data. Loading data...
api            INFO 	Fetching position data...
api            INFO 	Parsing position data...
api            INFO 	Data has been written to cache!
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loadi

## Driver selection 
I have choose Alsonso 

In [7]:
driver = 'ALO'

## Extraction data information
* Each row is one lap 

### FastF1

In [8]:
laps = session.load_laps(with_telemetry=True)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']


In [9]:
""" 
# Get all drivers
drivers = pd.unique(laps['Driver'])

telemetry = pd.DataFrame()

# Telemetry can only be retrieved driver-by-driver
for driver in drivers:
"""

" \n# Get all drivers\ndrivers = pd.unique(laps['Driver'])\n\ntelemetry = pd.DataFrame()\n\n# Telemetry can only be retrieved driver-by-driver\nfor driver in drivers:\n"

#### Create dataframe

In [10]:
data_driver = pd.DataFrame()

# Get all drivers
drivers = pd.unique(laps['Driver']) 

drivers = ['ALO'] #! TODO for all the drivers
for driver in drivers:

    driver_laps = laps.pick_driver(driver)

    rows_to_concat = []
    for lap in driver_laps.iterlaps():
        #FastF1 data
        #! TODO this will change in the bucle for year and race
        year = year #todo 
        round = round #todo

        race_name = session.event.EventName
        driver_number = lap[1].DriverNumber
        driver_name = lap[1].Driver
        team = lap[1].Team
        grid_position = int(session.get_driver(driver).GridPosition)
        qualy_position = 0  # TODO #Qualy finish position #TODO Sprints 

        #Data that changes every lap
        lap_number = lap[1].LapNumber
        compound = lap[1].Compound
        tyre_life = int(lap[1].TyreLife)
        num_pit_stops = int(lap[1].Stint) - 1 
        track_status = lap[1].TrackStatus
        #If it has change the tire tipes, O if not 1 if yes 
        if (not pd.DataFrame(rows_to_concat).empty and pd.DataFrame(rows_to_concat)['Compound'].nunique() >1 ):
            tyres_change= 1
        else: 
            tyres_change= 0

        #Creation of the new row
        new_row = {'Year': year, 'RaceNumber': round, 'RaceName': race_name, 'DriverNumber': driver_number,
                'Driver': driver_name, 'Team': team, 'GridPosition': grid_position,
                'QualyPositon': qualy_position, 'LapNumber': lap_number, 'Compound': compound,
                'TyreLife': tyre_life, 'NumberPitStops': num_pit_stops, 'Tyres_change': tyres_change,
                'TrackStatus': track_status}
        
        rows_to_concat.append(new_row)

    #is concat beacuse append with be deprecated in a future version  
    data_driver = pd.concat([data_driver, pd.DataFrame(rows_to_concat)], ignore_index=True)

data_driver.head()

,Year,RaceNumber,RaceName,DriverNumber,Driver,Team,GridPosition,QualyPositon,LapNumber,Compound,TyreLife,NumberPitStops,Tyres_change,TrackStatus
0,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,1,SOFT,4,0,0,2
1,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,2,SOFT,5,0,0,2
2,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,3,SOFT,6,0,0,1
3,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,4,SOFT,7,0,0,1
4,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,5,SOFT,8,0,0,1


### Ergast

In [11]:
# This is necessary because in the lap table we have the user id, and in fastF1 what we have is the code 
dict_code_drivers ={}

for year in list(range(year_start,year_finish+1)):
    url = 'http://ergast.com/api/f1/{}/drivers.json?limit=100000'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['DriverTable']['Drivers']:
        if (item['driverId'] not in dict_code_drivers):
            dict_code_drivers[item['driverId']] = item['code']

#### Get qualy results

In [12]:
url = 'http://ergast.com/api/f1/{}/{}/qualifying.json?limit=100000'
r = requests.get(url.format(year,round))
json = r.json()

results_qualy = {}
for item in json['MRData']['RaceTable']['Races'][0]['QualifyingResults']:
    results_qualy[item['Driver']['code']] = item['position']
    

#### Get position in championships before starting the race

In [13]:
#? TODO is this useful ? and the tams championship

#In the fist race there is no results 
drivers_standings = {}
points_championship = {}


if (round-1 >0): 
    
    url = 'http://ergast.com/api/f1/{}/{}/driverStandings.json?limit=100000'
    r = requests.get(url.format(year,round-1))
    json = r.json()

    for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']: 
        drivers_standings[item['Driver']['code']] = item['position']
        points_championship[item['Driver']['code']] = item['points']


#### Race Status (if finish or has a problem)
Latter to delete all the driver that doesn't finish

In [14]:
results_status = {}
url = 'http://ergast.com/api/f1/{}/{}/results.json?limit=100000'
r = requests.get(url.format(year,round))
json = r.json()

for item in json['MRData']['RaceTable']['Races'][0]['Results']: 
    results_status[item['Driver']['code']] = item['status']


#### Create de dataframe with the data
#### Calculate the position, time lap and time in the race of each driver

In [15]:
laps_position = pd.DataFrame()
numberLaps  = max(driver_laps['LapNumber'])  #TODO
# dictionary to store total lap times for each driver
driver_lap_times = {}

for lapN in range(1, numberLaps+1):
    url = 'http://ergast.com/api/f1/{}/{}/laps/{}.json?limit=100000'
    r = requests.get(url.format(year, round, lapN)) #TODO year and round will cange
    json = r.json()

    for item in json['MRData']['RaceTable']['Races'][0]['Laps'][0]['Timings']:
        driver = dict_code_drivers.get(item['driverId'])
        qualyPosition = results_qualy.get(driver)
        position = item['position']
        lap_time = datetime.strptime(item['time'], '%M:%S.%f').time()

        resultStatus = results_status.get(driver)
        if len(drivers_standings) == 0 : #In the fist race there is no results 
            driverStandings= 0
            pointsChampionship= 0
        else:
            driverStandings= drivers_standings.get(driver)
            pointsChampionship = points_championship.get(driver)

        # get total lap time for the current driver
        if driver in driver_lap_times:
            total_time = driver_lap_times[driver] + timedelta(minutes=lap_time.minute, seconds=lap_time.second, microseconds=lap_time.microsecond)
        else:
            total_time = timedelta(minutes=lap_time.minute, seconds=lap_time.second, microseconds=lap_time.microsecond)

        # update the dictionary with the total lap time for the current driver
        driver_lap_times[driver] = total_time
        
       
        # append the lap data to the dataframe
        laps_position = laps_position.append({'LapNumber': lapN, 'Driver': driver, 'ResultStatus':resultStatus, 
                                'QualyPosition': qualyPosition, 'DriverStandings' :driverStandings, 
                                'PointsChampionship':pointsChampionship, 'Position': position, 
                                'TimeLap': lap_time, 'RaceTimeProgress': total_time 
                                }, ignore_index=True)

laps_position


,LapNumber,Driver,ResultStatus,QualyPosition,DriverStandings,PointsChampionship,Position,TimeLap,RaceTimeProgress
0,1,VER,Finished,1,0,0,1,00:01:39.019000,0 days 00:01:39.019000
1,1,LEC,Engine,3,0,0,2,00:01:40.230000,0 days 00:01:40.230000
2,1,PER,Finished,2,0,0,3,00:01:41.063000,0 days 00:01:41.063000
3,1,SAI,Finished,4,0,0,4,00:01:41.659000,0 days 00:01:41.659000
4,1,HAM,Finished,7,0,0,5,00:01:42.288000,0 days 00:01:42.288000
...,...,...,...,...,...,...,...,...,...
1050,57,RUS,Finished,6,0,0,7,00:01:37.767000,0 days 01:34:52.609000
1051,57,BOT,Finished,12,0,0,8,00:01:38.407000,0 days 01:35:09.383000
1052,57,GAS,Finished,20,0,0,9,00:01:38.147000,0 days 01:35:10.489000
1053,57,ALB,Finished,15,0,0,10,00:01:38.137000,0 days 01:35:26.510000


In [16]:
laps_position

,LapNumber,Driver,ResultStatus,QualyPosition,DriverStandings,PointsChampionship,Position,TimeLap,RaceTimeProgress
0,1,VER,Finished,1,0,0,1,00:01:39.019000,0 days 00:01:39.019000
1,1,LEC,Engine,3,0,0,2,00:01:40.230000,0 days 00:01:40.230000
2,1,PER,Finished,2,0,0,3,00:01:41.063000,0 days 00:01:41.063000
3,1,SAI,Finished,4,0,0,4,00:01:41.659000,0 days 00:01:41.659000
4,1,HAM,Finished,7,0,0,5,00:01:42.288000,0 days 00:01:42.288000
...,...,...,...,...,...,...,...,...,...
1050,57,RUS,Finished,6,0,0,7,00:01:37.767000,0 days 01:34:52.609000
1051,57,BOT,Finished,12,0,0,8,00:01:38.407000,0 days 01:35:09.383000
1052,57,GAS,Finished,20,0,0,9,00:01:38.147000,0 days 01:35:10.489000
1053,57,ALB,Finished,15,0,0,10,00:01:38.137000,0 days 01:35:26.510000


#### Calculate the distance between the driver ahead and behind,and the distance to the leader

In [17]:
#Diference with the car ahead 
laps_position['NextTime'] = laps_position.groupby('LapNumber')['RaceTimeProgress'].shift(1)
laps_position['TimeDiffAhead'] = laps_position['RaceTimeProgress']  - laps_position['NextTime']  
laps_position['TimeDiffAhead'] = laps_position['TimeDiffAhead'].fillna(value=pd.Timedelta(0)) #The first driver does't have diference with others
laps_position = laps_position.drop(columns=['NextTime'])

In [18]:
#Diference with the car behind 
laps_position['NextTime'] = laps_position.groupby('LapNumber')['RaceTimeProgress'].shift(-1)
laps_position['TimeDiffBehind'] =  laps_position['NextTime']  - laps_position['RaceTimeProgress']  
laps_position['TimeDiffBehind'] = laps_position['TimeDiffBehind'].fillna(value=pd.Timedelta(0)) #The first driver does't have diference with others
laps_position = laps_position.drop(columns=['NextTime'])


In [19]:
#Diference with the leader

grouped = laps_position.groupby('LapNumber')

# calculate leader's race time progress in each group
leader_progress = grouped['RaceTimeProgress'].transform(lambda x: x[x.idxmin()])

# calculate difference to leader for each driver in each group
laps_position['TimeDiffToLeader'] = laps_position['RaceTimeProgress'] - leader_progress

# remove the RaceTimeProgress colum 
laps_position = laps_position.drop(columns=['RaceTimeProgress'])

laps_position.tail(12)

,LapNumber,Driver,ResultStatus,QualyPosition,DriverStandings,PointsChampionship,Position,TimeLap,TimeDiffAhead,TimeDiffBehind,TimeDiffToLeader
1043,56,ZHO,+1 Lap,13,0,0,16,00:01:33.996000,0 days 00:00:15.738000,0 days 00:00:00,0 days 00:02:16.022000
1044,57,VER,Finished,1,0,0,1,00:01:36.373000,0 days 00:00:00,0 days 00:00:11.987000,0 days 00:00:00
1045,57,PER,Finished,2,0,0,2,00:01:36.558000,0 days 00:00:11.987000,0 days 00:00:26.650000,0 days 00:00:11.987000
1046,57,ALO,Finished,5,0,0,3,00:01:37.422000,0 days 00:00:26.650000,0 days 00:00:09.415000,0 days 00:00:38.637000
1047,57,SAI,Finished,4,0,0,4,00:01:38.021000,0 days 00:00:09.415000,0 days 00:00:02.925000,0 days 00:00:48.052000
1048,57,HAM,Finished,7,0,0,5,00:01:38.255000,0 days 00:00:02.925000,0 days 00:00:03.525000,0 days 00:00:50.977000
1049,57,STR,Finished,8,0,0,6,00:01:37.991000,0 days 00:00:03.525000,0 days 00:00:01.371000,0 days 00:00:54.502000
1050,57,RUS,Finished,6,0,0,7,00:01:37.767000,0 days 00:00:01.371000,0 days 00:00:16.774000,0 days 00:00:55.873000
1051,57,BOT,Finished,12,0,0,8,00:01:38.407000,0 days 00:00:16.774000,0 days 00:00:01.106000,0 days 00:01:12.647000
1052,57,GAS,Finished,20,0,0,9,00:01:38.147000,0 days 00:00:01.106000,0 days 00:00:16.021000,0 days 00:01:13.753000


#### Pit stops

In [20]:
pitStops = pd.DataFrame()
url = 'http://ergast.com/api/f1/{}/{}/pitstops.json?limit=100000'
r = requests.get(url.format(year,round))
json = r.json()

for item in json['MRData']['RaceTable']['Races'][0]['PitStops']: 
    
    driver = dict_code_drivers.get(item['driverId'])
    lap = item['lap']
    stopNumber = item['stop']
    duration = item['duration']
    
    pitStops = pitStops.append({'driver': driver, 'lap': lap, 'stopNumber': stopNumber, 'duration': duration}, ignore_index=True)


In [21]:
#Numeric
laps_position["TimePit"] = 0
laps_position["NumberPitStops"] = 0

#binari 0 no 1 yes
#If the car has stop is this stint 
laps_position["HasPitLap"] = 0

#Pit, time,number of pits
for index, row in laps_position.iterrows():     
    
    value_pit = pitStops[(pitStops['driver'] == row['Driver']) & (pitStops['lap'] == str(row['LapNumber']))]
    if not value_pit.empty:
    #There is a pit stop 
        laps_position.loc[index, 'HasPitLap'] = 1
        laps_position.loc[index, 'TimePit'] = value_pit['duration'].item()
        #The number of stops on the previous lap is displayed, if there has been a pit stop this number is increased. 
        if (row['LapNumber']>1):
            laps_position.loc[index, 'NumberPitStops'] = laps_position.loc[(laps_position['Driver'] == row['Driver']) & (laps_position['LapNumber'] == (row['LapNumber']-1)), 'NumberPitStops'].item()+1
    elif (row['LapNumber']>1): 
        laps_position.loc[index, 'NumberPitStops'] = laps_position.loc[(laps_position['Driver'] == row['Driver']) & (laps_position['LapNumber'] == (row['LapNumber']-1)), 'NumberPitStops'].item()
    

In [22]:
#Creation of columns with the car ahead and behind, 
#We have nulls, whether it is the first or the last one. 
#? TODO delete them 

laps_position['DriverAhead'] = laps_position.groupby('LapNumber')['Driver'].shift(1)
laps_position['DriverBehind'] = laps_position.groupby('LapNumber')['Driver'].shift(-1)

In [23]:
#Calculation if the driver ahead an behind has stop
#It is done in another for, because I need the values that are calculated in the previous one

#binari 0 no 1 yes
laps_position["CarAheadPit"] = 0
laps_position["CarBehindPit"] = 0

for index, row in laps_position.iterrows():  
    #Has the driver ahead do a pit stop 
    #If the driver ahead is Nan is because is the fist one 
    if type(row['DriverAhead']) == str: ## if is not str is when is Nan 
        n_pits_car_ahead = laps_position.loc[(laps_position['Driver'] == row['DriverAhead']) & (laps_position['LapNumber'] == (row['LapNumber']))]['NumberPitStops'].item()
        if (n_pits_car_ahead > row['NumberPitStops']):
            laps_position.loc[index, 'CarAheadPit'] = 1

    #Has the driver behind do a pit stop 
    #If the driver behind is Nan is because is the last one 
    if type(row['DriverBehind']) == str: ## if is not str is when is Nan 
        n_pits_car_ahead = laps_position.loc[(laps_position['Driver'] == row['DriverBehind']) & (laps_position['LapNumber'] == (row['LapNumber']))]['NumberPitStops'].item()
        if (n_pits_car_ahead > row['NumberPitStops']):
            laps_position.loc[index, 'CarBehindPit'] = 1
    

### Was a good pit stop

In [24]:
# TODO look for a better way 

#0 no 1 yes
laps_position["GoodPitStop"] = 0

for index, row in laps_position.iterrows():  
    #if in this lap there is a pit stop 
    if row["HasPitLap"] ==1 :
        posInicial = row['Position']
        #Check if in this stint he has improved his position or stayed the same (with that you can say it is a good pit)
        for l in range(row['LapNumber'] +1 , numberLaps+1):
            
            #New laps to compare to the original 
            compare = laps_position.loc[(laps_position['Driver'] == row['Driver']) & (laps_position['LapNumber'] == l)]
            
            if ( not compare.empty): #check that there is information for this lap, if the car ends before 
                if (row['NumberPitStops'] != compare['NumberPitStops'].item()): #change of stint
                    if (row['Position'] >= compare['Position'].item()): #compare if the new position is better or the same
                        laps_position.loc[index, 'GoodPitStop'] = 1
                    break #we have the information that we want

                if (l == numberLaps): #last lap, is also an end of stint 
                    if (row['Position'] >= compare['Position'].item()): #compare if the new position is better or the same
                        laps_position.loc[index, 'GoodPitStop'] = 1


In [25]:
laps_position

,LapNumber,Driver,ResultStatus,QualyPosition,DriverStandings,PointsChampionship,Position,TimeLap,TimeDiffAhead,TimeDiffBehind,TimeDiffToLeader,TimePit,NumberPitStops,HasPitLap,DriverAhead,DriverBehind,CarAheadPit,CarBehindPit,GoodPitStop
0,1,VER,Finished,1,0,0,1,00:01:39.019000,0 days 00:00:00,0 days 00:00:01.211000,0 days 00:00:00,0,0,0,NaN,LEC,0,0,0
1,1,LEC,Engine,3,0,0,2,00:01:40.230000,0 days 00:00:01.211000,0 days 00:00:00.833000,0 days 00:00:01.211000,0,0,0,VER,PER,0,0,0
2,1,PER,Finished,2,0,0,3,00:01:41.063000,0 days 00:00:00.833000,0 days 00:00:00.596000,0 days 00:00:02.044000,0,0,0,LEC,SAI,0,0,0
3,1,SAI,Finished,4,0,0,4,00:01:41.659000,0 days 00:00:00.596000,0 days 00:00:00.629000,0 days 00:00:02.640000,0,0,0,PER,HAM,0,0,0
4,1,HAM,Finished,7,0,0,5,00:01:42.288000,0 days 00:00:00.629000,0 days 00:00:00.374000,0 days 00:00:03.269000,0,0,0,SAI,RUS,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,57,RUS,Finished,6,0,0,7,00:01:37.767000,0 days 00:00:01.371000,0 days 00:00:16.774000,0 days 00:00:55.873000,0,2,0,STR,BOT,0,0,0
1051,57,BOT,Finished,12,0,0,8,00:01:38.407000,0 days 00:00:16.774000,0 days 00:00:01.106000,0 days 00:01:12.647000,0,2,0,RUS,GAS,0,1,0
1052,57,GAS,Finished,20,0,0,9,00:01:38.147000,0 days 00:00:01.106000,0 days 00:00:16.021000,0 days 00:01:13.753000,0,3,0,BOT,ALB,0,0,0
1053,57,ALB,Finished,15,0,0,10,00:01:38.137000,0 days 00:00:16.021000,0 days 00:00:01.096000,0 days 00:01:29.774000,0,3,0,GAS,TSU,0,0,0


## Combine dataframes

In [26]:
data_driver = pd.merge(data_driver, laps_position, on=['LapNumber', 'Driver'])
data_driver.head()

,Year,RaceNumber,RaceName,DriverNumber,Driver,Team,GridPosition,QualyPositon,LapNumber,Compound,...,TimeDiffBehind,TimeDiffToLeader,TimePit,NumberPitStops_y,HasPitLap,DriverAhead,DriverBehind,CarAheadPit,CarBehindPit,GoodPitStop
0,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,1,SOFT,...,0 days 00:00:00.546000,0 days 00:00:04.589000,0,0,0,RUS,BOT,0,0,0
1,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,2,SOFT,...,0 days 00:00:00.710000,0 days 00:00:06.407000,0,0,0,RUS,BOT,0,0,0
2,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,3,SOFT,...,0 days 00:00:01.007000,0 days 00:00:07.501000,0,0,0,RUS,BOT,0,0,0
3,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,4,SOFT,...,0 days 00:00:01.282000,0 days 00:00:09.073000,0,0,0,RUS,BOT,0,0,0
4,2023,1,Bahrain Grand Prix,14,ALO,Aston Martin,5,0,5,SOFT,...,0 days 00:00:01.366000,0 days 00:00:10.700000,0,0,0,RUS,STR,0,0,0
